# Creating a task in Dime.Scheduler with the SDK

Check out the [📚 docs »](https://sdk.dimescheduler.com) for more information on the SDK. For everything else related to Dime.Scheduler, check out the [📚 official docs](https://docs.dimescheduler.com/).

## What we will do

This notebook is an interactive guide that will you show you to create a task in Dime.Scheduler. 

## What is a task

A **task** is a unit of work that needs to be scheduled, and as such it could be anything:

- Send a technician to the customer to fix the broken A/C
- Assign a consultant to a project that you just sold
- Manufacture a batch of orange juice
- Assemble a bicycle
- Rent out a piece of equipment

**Tasks** are the low-level units of work, and they always belong to a **job**, which is the Dime.Scheduler equivalent of a service order, project, assembly order, sales order, etc. Jobs container master data such as customer information, contact information, addresses, etc.

In what follows is a step-by-step guide that shows you how to use the SDK to create jobs and tasks. 

> We'll assume that a Dime.Scheduler instance is up and running, and accessible from the machine you're currently working on.

## Step 1: Retrieving the bits and bytes

All logic to interact with Dime.Scheduler is contained in the `Dime.Scheduler.Sdk` NuGet package, which is available [here](https://www.nuget.org/packages/Dime.Scheduler.Sdk/). The instruction below will fetch this package, which we will invoke once it's been downloaded.

In [1]:
#r "nuget:Dime.Scheduler.Sdk, 1.0.0-beta.12"

Installed Packages Dime.Scheduler.Sdk, 1.0.0-beta.12

## Step 2: Adding references to the SDK

Now that the package is loaded, we can start having some fun.  We'll start by adding a reference to the SDK in the code, so the compiler knows what we're talking about:

In [1]:
using Dime.Scheduler.Sdk;

## Step 3: Credentials

A regular Dime.Scheduler forms account with a **user name** (i.e. email addresss) and a **password** is enough to connect with Dime.Scheduler. An obviously we'll need the **URI** where your Dime.Scheduler instance is hosted.
Then we'll construct an instance of the `FormsAuthenticator` class, which takes the three pieces of data marked in bold.

In [1]:
string uri = "http://ds-vnext-test.azurewebsites.net/";
string user = "hendrik@dimescheduler.com";
string pw =  "Admin!1";
IAuthenticator authenticator = new FormsAuthenticator(uri, user, pw);

## Step 4: Accessing the endpoint

At this point, no contact has been made yet with Dime.Scheduler.  The entry point of the SDK is the `DimeSchedulerClient` class, which takes two parameters: the URI to Dime.Scheduler and an instance of `IAuthenticator`.
The `DimeSchedulerClient` gives you access to all the supported endpoints. By far, the most important endpoint is the `Import` endpoint.

When you invoke the `Request` method, this will start an instance of that endpoint and open the context. In other words, it will open up an authenticated line with Dime.Scheduler and you'll be able to transfer data securely to the planning tool. If you have entered invalid credentials or an inaccessible URI, you will get an exception here. If not, you may assume all is well.

In [1]:
DimeSchedulerClient client = new(uri, authenticator);
IImportEndpoint importEndpoint = await client.Import.Request();

## Step 5: Constructing the request

The `Dime.Scheduler.Sdk.Import` namespace holds all the models which you may insert in the `Import` endpoint. Essentially, any class that imports the `IImportRequestable` interface is accepted by the import pipeline.

As mentioned at the beginning of this notebook, to create task, we actually need to provide two entities:

- The **job**, which holds a set of tasks. Think of a job as a project and a task of one of the many activities that need to be done to finish the project
- The *task* itself, which must refer to a job that is already available in Dime.Scheduler

A job, therefore, is not always necessary. If the job is already available, you may skip step 5A, but make sure to use the correct `JobNo` value in step 5B.

### Step 5A: Creating a job

The job model contains a lot of information. So much that we've created a `JobBuilder` class to facilitate the creation of these objects. 
Most of the fields speak for themselves, although we should highlight the following three fields:

- **Source App**: this is a unique identifier that tells to which back-office system this job (and the subsequent tasks) belongs to.
- **Source Type**: this is a unique identifier that tells the underlying back-office system what module this job (and the subsequent tasks) belongs to
- **Job No**: this is a unique identifier which must exist in Dime.Scheduler before a task can be created.

In [1]:
using Dime.Scheduler.Sdk.Import;
using Dime.Scheduler.Sdk.Import.Builder;

string sourceApp = "NOTEBOOK";
string sourceType = "SERVICE";

Job newJobToCreate = new JobBuilder()
    .WithJob(sourceApp, sourceType, "NB0001", "Yearly maintenance & repair job", "Yearly service", "Simon Pecker")
    .WithCustomer(new Customer
    {
        Email = "hq@customer.com",
        Name = "HQ London",
        No = "HQ_LONDON",
        Phone = "123 423 091",
        Reference = "REF"
    })
    .Create();


The last step is to invoke the request and let the import pipeline of Dime.Scheduler do its job. To insert or update, use the `TransactionType.Append` value. To delete, you can specify ` TransactionType.Delete`. 

In [1]:
await importEndpoint.ProcessAsync(newJobToCreate, TransactionType.Append);

### Step 5B: Creating a task

Once the job is created, we can go ahead and link a task to that job. It is important that the source app, type, and job no link to the right job. And lastly, the **Task No** field is an unique identifier that can be used by both Dime.Scheduler and the back-office system so they understand this is the same task.

In [1]:
Task newTaskToCreate = new Task() 
{
	SourceApp = sourceApp,
	SourceType = sourceType,
	JobNo = "NB0001", // There must be a job in the database that holds this value
	TaskNo = "NO0001_2022_ROOM6",
	Description= "Service air conditioning in room #6",
	ShortDescription= "Service air conditioning in room #6"
};

And finally, invoke the request:

In [1]:
await importEndpoint.ProcessAsync(newTaskToCreate, TransactionType.Append);

So there you go, you've just created a task in Dime.Scheduler and should be now available in the open task list. So now it's time to stop puzzling and start planning!